
# Barbero durmiente

Para tratar el problema se  ha implementado una clase que simula la Barberria con los procesos que se ejecutan dentro de ella.
Se proporciona el esqueleto de la implementación de los procesos para que se marquen las zonas de exclusión mutua, de espera o de sincronazación. 
En el constructor de la Barberia se inicilizan todos los cerrojos, semaforos, condiciones y variables que se necesitan para la implementación.

In [1]:
import threading
import time
import random

class Barberia:
    def __init__(self):
        #mecanismos y variables de control clientes y acceso a la sala
        ####### inicializa cada uno con el tipo y valor oportuno####
        #####puedes cambiar los nombres o añadir nuevos#######

        self.MAX_CLIENTES = 3 

        #controla la entrada en la sala de espera
        self.entrarEnSala = threading.Semaphore(3)
        #controla el número de clientes en la sala
        self.clientesEnSala = 0 
        #garantiza exclusion para modificacion de variables de control de sala o acceso a secciones
        self.exclusionMutuaSala = threading.Lock()
        
        #mecanismos y variables de contrrol del barbero
        ####### inicializa cada uno con el tipo y valor oportuno #######
        #####puedes cambiar los nombres o añadir nuevos#######
        #controla el acesso al sillon de corte, puede poner clientes en espera y ser despertados
        self.sillon = threading.Condition()
        #controla lel estado del barbero
        self.barberoOcupado = False
        #puede poner al barbero a dormir y ser despertado
        self.dormir = threading.Condition()
        

    def actividadBarbero(self):
        while True:
            ### INCLUYE EL CÓDIGO NECESARIO PARA PROTOCOLO INICIAL   ###
            ### consultar en exclusion mutua los clientes que hay en la sala###
            with self.dormir:
                print ("\nB: Clientes en sala: " + str(self.clientesEnSala))
                if self.clientesEnSala < 1:
                    ### si no hay clientes esperando se pone a dormir hasta que le despierten ####
                    print ("B: El barbero se duerme.")
                    self.dormir.wait()
                
            print("B: El barbero da paso a un cliente")
            ###  INTRODUCE EL CÓDIGO PARA CAMBIAR EL ESTADO DEL BARBERO ###
            with self.exclusionMutuaSala:
                self.barberoOcupado = True
                print("B: El barbero pasa a estar ocupado, listo para cortar el pelo.")
            
            with self.sillon:
                self.sillon.notify()
                print("B: El barbero tiene el sillón libre. Esperando que se siente alguien.")

            time.sleep(5)
            

            ###  INTRODUCE EL CÓDIGO PARA CAMBIAR EL ESTADO DEL BARBERO Y AVISAR A LOS CLIENTES DE LA SALA ###
            with self.exclusionMutuaSala:
                self.barberoOcupado = False
                print("B: El barbero ya vuelve a estar libre.")
                    


            
    def actividadCliente(self):

        print("C: El "+threading.current_thread().name+" llega a la puerta de la barbería.")
        ### INCLUYE EL CÓDIGO NECESARIO PARA PROTOCOLO INICIAL   ###
        
        
        with self.exclusionMutuaSala:
            #print ("C1 ")

            ### consultar en exclusion mutua el estado del barbero###
            if self.barberoOcupado:
                #print ("C2 ")
                # Solo intentamos adquirir el semáforo si sabemos que debemos esperar.
                ### según estado entrar en la sala o no ####
                print("C: El " + threading.current_thread().name + " revisa si hay sitio. Espera a poder entrar.")
                #self.entrarEnSala.acquire()
                #print("C: El " + threading.current_thread().name + " entra en la sala.")

                if self.clientesEnSala >= self.MAX_CLIENTES:
                    # Si la sala está llena, el cliente se va sin esperar.
                    print("C: La sala está llena, " + threading.current_thread().name + " se va sin pelarse.")
                    return
                #else:
                    #self.clientesEnSala += 1
                    #print ("C: El " + threading.current_thread().name + " entra en la sala de espera.")

            
                # Adquirir el semáforo fuera de la exclusión mutua para evitar deadlocks. (Aunque en realidad... para qué me vale? Si ya controlo las sillas con el número de clientes en sala...)
                #self.entrarEnSala.acquire()

                self.clientesEnSala += 1
                print ("C: El " + threading.current_thread().name + " entra en la peluquería (al peluquero o a la sala).")
                print ("C: La sala/peluquería tiene uno más. Clientes en la sala: " + str(self.clientesEnSala))
    
        
        with self.dormir:
            ### según estado entrar despertar al babero o no ####
            #with self.exclusionMutuaSala:
                #if not self.barberoOcupado:
            if self.clientesEnSala == 0:
                #print ("C4 ")
                print ("C: El " + threading.current_thread().name + " despierta al barbero.")
                self.dormir.notify()
        

        
        #print ("C5 ")
        print ("C: El " + threading.current_thread().name + " está esperando para pelarse.")
        with self.sillon:
            self.sillon.wait()
            print ("C: Está en el sillón el "+threading.current_thread().name)


        with self.exclusionMutuaSala:
            if self.clientesEnSala > 0:
                self.clientesEnSala -= 1 

        #self.entrarEnSala.release()  # Libera el semáforo al salir de la sala de espera.
        print ("C: El cliente se libera si estaba ocupado. Clientes en la sala: " + str(self.clientesEnSala))
            
        time.sleep(3)

        #self.sillon.release()
        print ("C: El "+threading.current_thread().name+" abandona la barbería pelado.")


if __name__=="__main__":

        barberia = Barberia()
        
        ### INCLUYE EL CÓDIGO NECESARIO PARA CREAR E INICIALIZAR LOS HILOS  ###
       
        # Crear e iniciar el hilo del barbero
        barbero = threading.Thread(target = barberia.actividadBarbero)
        barbero.start()
        #barbero.join()
        
        ### clientes####
        # Crear e iniciar hilos para varios clientes
        for i in range(20):
            cliente = threading.Thread(target = barberia.actividadCliente, name=f'Cliente {i}')
            time.sleep(random.uniform(2, 4))  # Simula la llegada de clientes en momentos aleatorios
            cliente.start()
            #cliente.join()




B: Clientes en sala: 0
B: El barbero se duerme.
C: El Cliente 0 llega a la puerta de la barbería.
C: El Cliente 0 despierta al barbero.
C: El Cliente 0 está esperando para pelarse.
B: El barbero da paso a un cliente
B: El barbero pasa a estar ocupado, listo para cortar el pelo.
B: El barbero tiene el sillón libre. Esperando que se siente alguien.
C: Está en el sillón el Cliente 0
C: El cliente se libera si estaba ocupado. Clientes en la sala: 0
C: El Cliente 1 llega a la puerta de la barbería.
C: El Cliente 1 revisa si hay sitio. Espera a poder entrar.
C: El Cliente 1 entra en la peluquería (al peluquero o a la sala).
C: La sala/peluquería tiene uno más. Clientes en la sala: 1
C: El Cliente 1 está esperando para pelarse.
C: El Cliente 0 abandona la barbería pelado.
C: El Cliente 2 llega a la puerta de la barbería.
C: El Cliente 2 revisa si hay sitio. Espera a poder entrar.
C: El Cliente 2 entra en la peluquería (al peluquero o a la sala).
C: La sala/peluquería tiene uno más. Clientes 

C: El Cliente 13 abandona la barbería pelado.
B: El barbero ya vuelve a estar libre.

B: Clientes en sala: 3
B: El barbero da paso a un cliente
B: El barbero pasa a estar ocupado, listo para cortar el pelo.
B: El barbero tiene el sillón libre. Esperando que se siente alguien.
C: Está en el sillón el Cliente 15
C: El cliente se libera si estaba ocupado. Clientes en la sala: 2
C: El Cliente 15 abandona la barbería pelado.
B: El barbero ya vuelve a estar libre.

B: Clientes en sala: 2
B: El barbero da paso a un cliente
B: El barbero pasa a estar ocupado, listo para cortar el pelo.
B: El barbero tiene el sillón libre. Esperando que se siente alguien.
C: Está en el sillón el Cliente 17
C: El cliente se libera si estaba ocupado. Clientes en la sala: 1
C: El Cliente 17 abandona la barbería pelado.
B: El barbero ya vuelve a estar libre.

B: Clientes en sala: 1
B: El barbero da paso a un cliente
B: El barbero pasa a estar ocupado, listo para cortar el pelo.
B: El barbero tiene el sillón libre. 